# Entrega Final SME0828 - Introdução à Ciência de Dados
Professor: Francisco A. Rodrigues


Izabela Camila Souza Silva - 13827570

# Introdução


O projeto consiste em um estudo de Ciência de Dados e Probabilidade focado na análise exploratória e modelagem preditiva do conjunto de dados de abalones (moluscos marinhos). O objetivo central é empregar técnicas avançadas para inferir a idade dos animais a partir de suas características físicas, além de abordar questões ecológicas e biológicas secundárias de grande valor prático.

Para o êxito dessa tarefa, foi utilizado a base de dados fornecida pela Universidade da California (UCI). A base contém diversas características físicas de abalones, com 8 covariáveis além da variável resposta (número de anéis).

# Objetivos



O dataset disponibilizado pela UCI tem como propósito inicial avaliar se é possível estimar a idade de moluscos por meio de suas características físicas.

Durante a análise, ficou evidente que parte das informações presentes no conjunto só pode ser obtida após a morte do animal. Isso levantou a oportunidade de explorar um problema adicional: verificar se esses atributos pós-morte podem ser inferidos a partir de medições simples e não invasivas.

Com esse foco, defini os seguintes objetivos de investigação:

* Estimar o peso da concha e das vísceras utilizando apenas o peso total e as dimensões externas do molusco.

* Determinar o sexo dos animais com base exclusivamente nas variáveis observáveis no conjunto de dados.

* Identificar se o indivíduo é imaturo ou adulto sem recorrer à análise interna dos órgãos reprodutivos.

# Configuração do ambiente

## Bibliotecas

In [ ]:
!pip install ucimlrepo -q

In [ ]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
import statsmodels.formula.api as smf
from sklearn.linear_model import LinearRegression, Lasso, Ridge
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.utils import resample
from ucimlrepo import fetch_ucirepo

import warnings
warnings.filterwarnings("ignore", category=UserWarning, module="sklearn.linear_model._coordinate_descent")

## Funções

In [ ]:
def create_dict_quality(_name: str, _y_test: np.array, _pred: np.array) -> dict:
    _dict = {
        "Tipo Regressão": _name,
        "EQM": mean_squared_error(_y_test, _pred),
        "REQM": np.sqrt(mean_squared_error(_y_test, _pred)),
        "R^2": r2_score(_y_test, _pred)
            }
    return _dict

def create_dict_betas(_type: str, _lower_bounds: np.array, _upper_bounds: np.array) -> dict:
  number_of_estimates = len(_lower_bounds)
  _dict = [{"Tipo Regressão": _type,
            "Beta": f"Beta {i}",
            "Limite Inferior": _lower_bounds[i],
            "Limite Superior": _upper_bounds[i]} for i in range(number_of_estimates)]
  return _dict

def create_dict_ic(_type: str, _metric: str, _bounds: tuple) -> dict:
  _dict = {
      "Tipo Regressão": _type,
      "Função Avaliação": _metric,
      "Limite Inferior": _bounds[0],
      "Limite Superior": _bounds[1]
  }
  return _dict

# Preparo do Dados

## Importando os dados

In [ ]:
abalone = fetch_ucirepo(id=1)

X = abalone.data.features
y = abalone.data.targets

X.info()
y.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4177 entries, 0 to 4176
Data columns (total 8 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Sex             4177 non-null   object 
 1   Length          4177 non-null   float64
 2   Diameter        4177 non-null   float64
 3   Height          4177 non-null   float64
 4   Whole_weight    4177 non-null   float64
 5   Shucked_weight  4177 non-null   float64
 6   Viscera_weight  4177 non-null   float64
 7   Shell_weight    4177 non-null   float64
dtypes: float64(7), object(1)
memory usage: 261.2+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4177 entries, 0 to 4176
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype
---  ------  --------------  -----
 0   Rings   4177 non-null   int64
dtypes: int64(1)
memory usage: 32.8 KB


## Limpeza dos dados

Durante a verificação inicial do dataset, surgiram alguns valores claramente incompatíveis com a realidade: extremamente altos, extremamente baixos ou simplesmente inválidos. Além de medições iguais a zero, o que invalida dimensões corporais, contrariando a natueza dos dados

In [ ]:
for col in list(X.columns):
  minimo = X[col].min()
  if minimo == 0:
    indice = X[col].idxmin()
    novo_min = X[col][X[col] != 0].min()
    X.loc[indice, col] = novo_min


## Parâmetros

In [ ]:
test_split = 0.2
rand_state = 1996

## Separação conjunto de dados

O conjunto original apresenta variáveis qualitativas nominais, logo, o processo de encoding delas é necessário. <br>
Durante esse passo, para evitar problemas com colinearidade, o nível "fêmea" foi excluído na hora da conversão de encoding.

In [ ]:
X_dummied = pd.get_dummies(X, columns = ['Sex'], drop_first = True)
X_dummied['Sex_I'] = X_dummied['Sex_I'].map( {False: 0, True: 1} )
X_dummied['Sex_M'] = X_dummied['Sex_M'].map( {False: 0, True: 1} )

X_train, X_test, y_train, y_test = train_test_split(X_dummied, y, test_size = test_split, random_state=rand_state)

# Regressão

Para fins didáticos, escolhi começar com a regressão linear, principalmente por seu caráter introdutório e pela facilidade de compreensão do método. No entanto, começar por esse modelo traz uma limitação importante: a regressão linear assume que a variável resposta segue uma distribuição normal, um pressuposto que, até o momento, ainda não foi verificado.


## Regressão Linear Múltipla (OLS), regularização de Lasso (L1) e regularização de Ridge (L2)

Para essa etapa, organizei o processo de modelagem criando pipelines separados para cada método de regressão considerado: OLS, Lasso e Ridge. Cada pipeline inclui duas fases: primeiro, a padronização das variáveis por meio do StandardScaler e, em seguida, o ajuste do respectivo modelo.

Para os modelos regularizados (Lasso e Ridge), defini um conjunto de possíveis valores para o hiperparâmetro alpha, variando em escalas de potência de 10. Com esse grid, utilizei GridSearchCV para identificar o valor de alpha que minimiza o erro quadrático médio através de validação cruzada. Esse procedimento foi realizado individualmente para Lasso e Ridge.

Após determinar o melhor alpha para cada um deles, treinei os modelos finais usando os parâmetros selecionados. O OLS foi ajustado diretamente, já que não possui hiperparâmetros equivalentes.

Em seguida, gerei as previsões no conjunto de teste para todos os modelos — OLS, Lasso e Ridge — e avaliei o desempenho de cada um usando uma função de métricas previamente definida. Por fim, os resultados foram organizados em um DataFrame para facilitar a comparação entre as abordagens.

In [ ]:
# Definição dos pipelines para cada modelo
pipelines = {
    "OLS": Pipeline([
        ("scaler", StandardScaler()),
        ("model", LinearRegression())
    ]),

    "Lasso": Pipeline([
        ("scaler", StandardScaler()),
        ("model", Lasso(max_iter=10000, tol=1e-4))
    ]),

    "Ridge": Pipeline([
        ("scaler", StandardScaler()),
        ("model", Ridge(max_iter=10000, tol=1e-4))
    ])
}

# Grid de hiperparâmetros para os modelos regulares
alphas = [10 ** (-i) for i in range(2, 20)]
grid = {"model__alpha": alphas}

# Ajuste dos hiperparâmetros para Lasso e Ridge
search_results = {}

for name in ["Lasso", "Ridge"]:
    search = GridSearchCV(
        estimator=pipelines[name],
        param_grid=grid,
        cv=5,
        scoring="neg_mean_squared_error"
    )
    search.fit(X_train, y_train)
    search_results[name] = search

# Exibição dos melhores valores de alpha
print("Melhor alpha (Lasso):", search_results["Lasso"].best_params_["model__alpha"])
print("Melhor alpha (Ridge):", search_results["Ridge"].best_params_["model__alpha"])

# Treinamento dos modelos finais
ols_model = pipelines["OLS"].fit(X_train, y_train)
lasso_model = search_results["Lasso"].best_estimator_
ridge_model = search_results["Ridge"].best_estimator_

# Predições para o conjunto de teste
predicoes = {
    "OLS": ols_model.predict(X_test),
    "Lasso": lasso_model.predict(X_test),
    "Ridge": ridge_model.predict(X_test)
}

# Avaliação dos modelos
resultado_df = pd.DataFrame([
    create_dict_quality(nome, y_test, predicoes[nome])
    for nome in predicoes
]).set_index("Tipo Regressão")

display(resultado_df)


Melhor alpha (Lasso): 0.001
Melhor alpha (Ridge): 1e-13


,EQM,REQM,R^2
Tipo Regressão,,,
OLS,4.640558,2.154195,0.535941
Lasso,4.638283,2.153667,0.536169
Ridge,4.640558,2.154195,0.535941


Após identificar os valores de alpha que melhor ajustam os modelos Lasso (10⁻³) e Ridge (10⁻¹²), ajustei tanto a regressão OLS quanto as versões regularizadas. Com todos os modelos treinados, avancei para a etapa de comparação entre eles.

A análise dos resultados mostra alguns pontos importantes:

• Os valores ótimos de alpha são muito pequenos, indicando que a regularização tem pouca influência. Isso sugere que o modelo sem penalização já representa bem a relação entre as variáveis.

• Os valores de R² obtidos pelos três métodos são muito próximos, porém relativamente baixos. Esse comportamento reforça a evidência observada na análise exploratória de que o conjunto de variáveis possui capacidade limitada de explicar a resposta.

• As métricas de erro (EQM e REQM) também são semelhantes entre os modelos, e seus valores são elevados. Isso confirma que, apesar das abordagens testadas, o desempenho preditivo permanece insatisfatório.

Esses resultados indicam que, para esse conjunto de dados, modelos lineares — regularizados ou não — apresentam limitações claras na tarefa de prever a variável alvo.


### Análise da regularização de Lasso (L1) e Ridge (L2) sobre o conjunto de dados:

Nesta etapa, foram construídos dois pipelines básicos para avaliar o desempenho das regressões Lasso e Ridge quando utilizam o valor padrão de alpha igual a 1.0. Cada pipeline inclui a padronização das variáveis com StandardScaler seguida do ajuste do respectivo modelo.

Depois de treinar ambos os modelos com o conjunto de treino, foram geradas as previsões no conjunto de teste. Em seguida, as métricas de avaliação foram calculadas para comparar o comportamento das versões base de Lasso e Ridge.

Os resultados obtidos permitem observar como cada método se comporta quando não há ajuste fino do hiperparâmetro de regularização, servindo como referência para comparação com os modelos otimizados.


In [ ]:
# Definição dos pipelines básicos para Lasso e Ridge (alpha = 1.0)
pipeline_base_lasso = Pipeline([
    ("normalizacao", StandardScaler()),
    ("modelo", Lasso(alpha=1.0))
])

pipeline_base_ridge = Pipeline([
    ("normalizacao", StandardScaler()),
    ("modelo", Ridge(alpha=1.0))
])

# Ajuste dos modelos utilizando os dados de treino
modelo_lasso_base = pipeline_base_lasso.fit(X_train, y_train)
modelo_ridge_base = pipeline_base_ridge.fit(X_train, y_train)

# Geração das predições no conjunto de teste
y_pred_lasso_base = modelo_lasso_base.predict(X_test)
y_pred_ridge_base = modelo_ridge_base.predict(X_test)

# Comparação do desempenho dos modelos
resultados_base = pd.DataFrame([
    create_dict_quality("Lasso", y_test, y_pred_lasso_base),
    create_dict_quality("Ridge", y_test, y_pred_ridge_base)
]).set_index("Tipo Regressão")

display(resultados_base)


,EQM,REQM,R^2
Tipo Regressão,,,
Lasso,6.991496,2.644144,0.300846
Ridge,4.637594,2.153507,0.536238


A comparação entre os modelos revela comportamentos bastante distintos quando se utiliza o valor padrão de regularização. No caso do Lasso, a alteração no parâmetro alpha impacta fortemente o desempenho: os erros aumentam de forma expressiva (aproximadamente 50%) enquanto o valor de $R^{2}$ sofre uma queda perceptível, reduzindo cerca de 0,2 ponto percentual em relação ao modelo otimizado.

Por outro lado, o Ridge apresenta um comportamento muito mais estável. Mesmo com o uso de alpha igual a 1.0, seus resultados permanecem praticamente inalterados, tanto nas métricas de erro quanto no coeficiente de determinação, indicando menor sensibilidade à variação desse hiperparâmetro.

### Intervalos de confiança Lasso (L1) e Ridge (L2)



O código realiza a construção e revisão de gráficos de controle X̄ e R de forma automática. A função executa as seguintes etapas:

1. Cálculo das estatísticas dos subgrupos  
   São obtidas a média (X̄) e a amplitude (R) de cada subgrupo, usadas para monitorar o comportamento do processo.

2. Cálculo dos limites de controle iniciais  
   Utilizam-se as constantes A2, D3 e D4 para determinar LSC, LC e LIC dos gráficos X̄ e R.

3. Detecção automática de pontos fora de controle  
   O código identifica subgrupos que ultrapassam qualquer limite, indicando possíveis causas especiais.

4. Revisão dos limites de controle
   Os subgrupos fora de controle são removidos e novas estatísticas (média, amplitude e limites) são recalculadas — simulando o procedimento de ajuste após remoção das causas atribuíveis.

5. Geração dos gráficos finais  
   São exibidos os gráficos X̄ e R com:
   - dados originais,
   - limites iniciais,
   - pontos fora de controle destacados,
   - limites revisados após a limpeza.

Este fluxo permite comparar o processo antes e depois da retirada de anomalias, avaliando a estabilidade estatística e a variabilidade real do sistema.


In [ ]:
def rodar_bootstrap(X, y, n_bootstraps=1000):


    coefs_ols   = np.zeros((n_bootstraps, X.shape[1]))
    coefs_lasso = np.zeros((n_bootstraps, X.shape[1]))
    coefs_ridge = np.zeros((n_bootstraps, X.shape[1]))

    r2_ols, mse_ols, rmse_ols = [], [], []
    r2_lasso, mse_lasso, rmse_lasso = [], [], []
    r2_ridge, mse_ridge, rmse_ridge = [], [], []

    ols_model   = LinearRegression()
    lasso_model = Lasso(alpha=0.001)
    ridge_model = Ridge(alpha=1 / 10**9)

    indices = np.arange(len(y))

    for i in range(n_bootstraps):

        # Bootstrap sample
        idx_bootstrap = resample(indices, random_state=i)
        idx_test = np.setdiff1d(indices, idx_bootstrap)

        X_train, y_train = X.iloc[idx_bootstrap], y.iloc[idx_bootstrap]
        X_test, y_test   = X.iloc[idx_test], y.iloc[idx_test]

        # Treinando modelos
        ols_model.fit(X_train, y_train)
        lasso_model.fit(X_train, y_train)
        ridge_model.fit(X_train, y_train)

        # Coeficientes
        coefs_ols[i]   = ols_model.coef_
        coefs_lasso[i] = lasso_model.coef_
        coefs_ridge[i] = ridge_model.coef_

        # Predições
        pred_ols   = ols_model.predict(X_test)
        pred_lasso = lasso_model.predict(X_test)
        pred_ridge = ridge_model.predict(X_test)

        # R²
        r2_ols.append(r2_score(y_test, pred_ols))
        r2_lasso.append(r2_score(y_test, pred_lasso))
        r2_ridge.append(r2_score(y_test, pred_ridge))

        # EQM
        mse_ols.append(mean_squared_error(y_test, pred_ols))
        mse_lasso.append(mean_squared_error(y_test, pred_lasso))
        mse_ridge.append(mean_squared_error(y_test, pred_ridge))

        # REQM
        rmse_ols.append(np.sqrt(mse_ols[-1]))
        rmse_lasso.append(np.sqrt(mse_lasso[-1]))
        rmse_ridge.append(np.sqrt(mse_ridge[-1]))

    betas = {
        "OLS":   (np.percentile(coefs_ols,   2.5, axis=0), np.percentile(coefs_ols,   97.5, axis=0)),
        "Lasso": (np.percentile(coefs_lasso, 2.5, axis=0), np.percentile(coefs_lasso, 97.5, axis=0)),
        "Ridge": (np.percentile(coefs_ridge, 2.5, axis=0), np.percentile(coefs_ridge, 97.5, axis=0))
    }

    tabela_betas = pd.concat([
        pd.DataFrame(create_dict_betas(nome, low, high)).set_index("Tipo Regressão")
        for nome, (low, high) in betas.items()
    ]).reset_index().pivot(
        index="Beta",
        columns="Tipo Regressão",
        values=["Limite Inferior", "Limite Superior"]
    )

    metricas = {
        "OLS":   (mse_ols, rmse_ols, r2_ols),
        "Lasso": (mse_lasso, rmse_lasso, r2_lasso),
        "Ridge": (mse_ridge, rmse_ridge, r2_ridge)
    }

    linhas_metricas = []
    for nome, (mse_list, rmse_list, r2_list) in metricas.items():
        linhas_metricas.append(create_dict_ic(nome, "EQM",  np.percentile(mse_list,  [2.5, 97.5])))
        linhas_metricas.append(create_dict_ic(nome, "REQM", np.percentile(rmse_list, [2.5, 97.5])))
        linhas_metricas.append(create_dict_ic(nome, "R²",   np.percentile(r2_list,   [2.5, 97.5])))

    tabela_metricas = pd.DataFrame(linhas_metricas).set_index("Tipo Regressão").reset_index().pivot(
        index="Função Avaliação",
        columns="Tipo Regressão",
        values=["Limite Inferior", "Limite Superior"]
    )


    return tabela_betas, tabela_metricas


In [ ]:
tabela_betas, tabela_metricas = rodar_bootstrap(X_dummied, y, n_bootstraps=1000)

display(tabela_betas)
display(tabela_metricas)


Limite Inferior                       Limite Superior  \
Tipo Regressão           Lasso        OLS      Ridge           Lasso   
Beta                                                                   
Beta 0                0.000000  -4.565493  -4.565493        3.255819   
Beta 1                5.045001   5.290763   5.290763       11.834680   
Beta 2                4.276405   5.043699   5.043699       23.162486   
Beta 3                5.916616   6.643774   6.643774       10.522360   
Beta 4              -21.654926 -22.546240 -22.546240      -16.480347   
Beta 5              -12.192599 -14.234260 -14.234260       -5.675418   
Beta 6                6.240090   5.307768   5.307768       12.538864   
Beta 7               -1.038832  -1.027771  -1.027771       -0.629904   
Beta 8               -0.122436  -0.124078  -0.124078        0.236761   

                                      
Tipo Regressão        OLS      Ridge  
Beta                                  
Beta 0           3.274621   3.274621  
Beta 1          15.656162  15.656162  
Beta 2          26.589722  26.589722  
Beta 3          11.408155  11.408155  
Beta 4         -17.154736 -17.154736  
Beta 5          -7.484522  -7.484522  
Beta 6          11.867990  11.867990  
Beta 7          -0.613571  -0.613571  
Beta 8           0.243417   0.243417

Limite Inferior                     Limite Superior  \
Tipo Regressão             Lasso       OLS     Ridge           Lasso   
Função Avaliação                                                       
EQM                     4.441320  4.445272  4.445272        5.410302   
REQM                    2.107444  2.108381  2.108381        2.326006   
R²                      0.486561  0.476334  0.476334        0.560858   

                                      
Tipo Regressão         OLS     Ridge  
Função Avaliação                      
EQM               5.503514  5.503514  
REQM              2.345957  2.345957  
R²                0.562015  0.562015

A partir das tabelas apresentadas, nota-se que a hipótese nula não é rejeitada sempre que o intervalo de confiança não inclui o valor 0.

Analisando os coeficientes β, apenas o intercepto ($\beta_{0}$) e o coeficiente $\beta_{8}$ apresentam intervalos que abrangem 0, indicando ausência de significância estatística nesses casos.

Já na tabela de métricas de avaliação, como esperado, nenhum dos intervalos inclui o valor 0. Vale destacar que esses intervalos são muito próximos entre si, sugerindo que o modelo conseguiu explorar quase totalmente a estrutura linear presente nos dados.

## Modelos lineares generalizados

Após esgotar as alternativas de modelos voltados para respostas contínuas, passei a investigar abordagens que tratam a variável dependente como uma contagem. Nessa nova etapa, a modelagem foi orientada por distribuições discretas, levando à avaliação de regressões lineares baseadas em:

* Distribuição de Poisson

* Distribuição Binomial Negativa

### Regressão Poisson & Binomial negativa Multipla


In [ ]:
# Adiciona constante às variáveis explicativas
X_const = sm.add_constant(X_dummied)

# Modelo de Poisson
poisson_model = sm.GLM(y, X_const, family=sm.families.Poisson()).fit()
print(poisson_model.summary())

y_pred_poisson = poisson_model.predict(X_const)
mse_poisson = mean_squared_error(y, y_pred_poisson)
print("Erro Quadrático Médio (Poisson):", mse_poisson)
print()

# Modelo de Binomial Negativa
nb_model = sm.GLM(y, X_const, family=sm.families.NegativeBinomial()).fit()
print(nb_model.summary())

y_pred_nb = nb_model.predict(X_const)
mse_nb = mean_squared_error(y, y_pred_nb)
print("Erro Quadrático Médio (Binomial Negativa):", mse_nb)


                 Generalized Linear Model Regression Results                  
Dep. Variable:                  Rings   No. Observations:                 4177
Model:                            GLM   Df Residuals:                     4167
Model Family:                 Poisson   Df Model:                            9
Link Function:                    Log   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -9484.9
Date:                Sat, 29 Nov 2025   Deviance:                       1835.1
Time:                        18:00:33   Pearson chi2:                 1.93e+03
No. Iterations:                     5   Pseudo R-squ. (CS):             0.4240
Covariance Type:            nonrobust                                         
                     coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------
const              1.4883      0.047     31.

/usr/local/lib/python3.12/dist-packages/statsmodels/genmod/families/family.py:1367: ValueWarning: Negative binomial dispersion parameter alpha not set. Using default value alpha=1.0.
  warnings.warn("Negative binomial dispersion parameter alpha not "



Os resultados mostram que nenhum dos modelos ajustados com distribuições discretas apresentou um EQM inferior aos modelos baseados na suposição de normalidade da variável resposta.

No caso do modelo de Poisson, os coeficientes $\beta_{1}$ e $\beta_{8}$ foram considerados não significativos. Já o modelo de Binomial Negativa apresentou não significância apenas para $\beta_{0}$ e $\beta_{5}$. Isso indica que, além de apresentarem ajuste inferior, os modelos atribuíram relevância estatística a conjuntos diferentes de variáveis.

Essa divergência é esperada, pois cada modelo opera sob premissas distintas sobre a distribuição da variável resposta, o que influencia diretamente o comportamento dos coeficientes e a qualidade do ajuste.


### Regressão considerando apenas comprimento e altura para prever os outros parâmetros dos animais.

O projeto parte de um interesse em compreender os dados dos abalones sob uma perspectiva ecológica. A motivação central é avaliar a possibilidade de prever características que hoje dependem da morte do animal, de forma a evitar esse procedimento sempre que possível. Isso traria benefícios tanto em termos de uso eficiente de recursos quanto em aspectos éticos ligados ao bem-estar animal.

Por exemplo, identificar previamente que um indivíduo possui grande proporção de peso concentrada na concha e nas vísceras e pouco peso de carne poderia evitar sua coleta desnecessária. Da mesma forma, a estimativa antecipada do estado de maturidade é relevante: se as dimensões físicas indicam que um abalone ainda não alcançou seu pleno desenvolvimento, ele pode ser mantido na natureza, evitando capturas prematuras de animais infants.

Essa abordagem reforça o propósito ecológico do estudo, buscando integrar conhecimento estatístico com práticas mais sustentáveis de manejo.

In [64]:
def rodar_regressoes_multialvo(X, y, test_split=0.2, rand_state=42):
    X_reduzido = X[['Length', 'Diameter', 'Height', 'Whole_weight']]
    targets = ['Shucked_weight', 'Viscera_weight', 'Shell_weight', 'Rings']

    # DataFrame expandido com novos y
    X_expandido = (
        X[['Shucked_weight', 'Viscera_weight', 'Shell_weight']]
        .reset_index()
        .merge(y.reset_index(), on="index", how="left")
        .drop(columns=['index'])
    )

    # Conjuntos de treino e teste para cada alvo
    conjuntos = {
        target: train_test_split(
            X_reduzido,
            X_expandido[[target]],
            test_size=test_split,
            random_state=rand_state
        )
        for target in targets
    }

    # Função para criar pipelines
    def make_pipeline(model):
        return Pipeline([
            ('scaler', StandardScaler()),
            ('regressor', model)
        ])

    # Pipelines
    ols_pipeline   = make_pipeline(LinearRegression())
    lasso_pipeline = make_pipeline(Lasso())
    ridge_pipeline = make_pipeline(Ridge())

    # Espaço de busca para o alpha
    param_grid = {'regressor__alpha': [10**(-exp) for exp in range(10, -2, -1)]}

    # Loop principal das análises
    for target in targets:
        print(f"Regressão para prever o valor de: {target}")

        X_train_r, X_test_r, y_train_r, y_test_r = conjuntos[target]

        # GridSearch Lasso
        lasso_search = GridSearchCV(
            lasso_pipeline, param_grid, cv=5, scoring='neg_mean_squared_error'
        )
        lasso_search.fit(X_train_r, y_train_r)

        # GridSearch Ridge
        ridge_search = GridSearchCV(
            ridge_pipeline, param_grid, cv=5, scoring='neg_mean_squared_error'
        )
        ridge_search.fit(X_train_r, y_train_r)

        # Exibir melhores alfas
        print("Melhor alfa Lasso:", lasso_search.best_params_['regressor__alpha'])
        print("Melhor alfa Ridge:", ridge_search.best_params_['regressor__alpha'])

        # Treino final
        ols_model  = ols_pipeline.fit(X_train_r, y_train_r)
        best_lasso = lasso_search.best_estimator_
        best_ridge = ridge_search.best_estimator_

        # Predições
        pred_ols   = ols_model.predict(X_test_r)
        pred_lasso = best_lasso.predict(X_test_r)
        pred_ridge = best_ridge.predict(X_test_r)

        # Avaliação
        resultados = [
            create_dict_quality("OLS", y_test_r, pred_ols),
            create_dict_quality("Lasso", y_test_r, pred_lasso),
            create_dict_quality("Ridge", y_test_r, pred_ridge)
        ]

        display(pd.DataFrame(resultados).set_index('Tipo Regressão'))
        print()


A função desenvolvida integra todas as etapas necessárias para a construção e análise de um Gráfico de Controle, incluindo:

* Cálculo dos limites de controle originais;

* Identificação automática dos pontos fora de controle;

* Remoção opcional desses pontos para obtenção dos limites revisados;

* Geração dos gráficos correspondentes;

* Retorno estruturado dos resultados numéricos.

A abordagem centraliza o processamento estatístico em uma única função parametrizável, permitindo reuso e permitindo incorporar a lógica tanto da Fase I (detecção de pontos atípicos para estudo da estabilidade) quanto da Fase II (monitoramento do processo).

In [ ]:
resultados = rodar_regressoes_multialvo(X, y)


Regressão para prever o valor de: Shucked_weight
Melhor alfa Lasso: 0.001
Melhor alfa Ridge: 10


,EQM,REQM,R^2
Tipo Regressão,,,
OLS,0.003281,0.057277,0.930735
Lasso,0.003323,0.057644,0.929846
Ridge,0.003282,0.057288,0.930709



Regressão para prever o valor de: Viscera_weight
Melhor alfa Lasso: 1e-05
Melhor alfa Ridge: 1e-10


,EQM,REQM,R^2
Tipo Regressão,,,
OLS,0.000784,0.028005,0.935522
Lasso,0.000784,0.028008,0.935506
Ridge,0.000784,0.028005,0.935522



Regressão para prever o valor de: Shell_weight
Melhor alfa Lasso: 1e-05
Melhor alfa Ridge: 1e-10


,EQM,REQM,R^2
Tipo Regressão,,,
OLS,0.001484,0.038518,0.919975
Lasso,0.001483,0.038511,0.920004
Ridge,0.001484,0.038518,0.919975



Regressão para prever o valor de: Rings
Melhor alfa Lasso: 0.1
Melhor alfa Ridge: 10


,EQM,REQM,R^2
Tipo Regressão,,,
OLS,6.724540,2.593172,0.378807
Lasso,6.806458,2.608919,0.371240
Ridge,6.722488,2.592776,0.378997


Os resultados obtidos pelas regressões são bastante consistentes e alinhados com o esperado, especialmente considerando a forte correlação linear previamente observada entre as variáveis preditoras. A validação estatística por meio dos modelos reforça a viabilidade da proposta inicial.

Entretanto, a variável que representa o número de anéis já reconhecida como pouco precisa mesmo quando todas as covariáveis são utilizadas simultaneamente apresenta desempenho ainda mais limitado quando a predição depende apenas de comprimento, diâmetro, altura e massa total. Isso evidencia a dificuldade inerente de estimar a idade do abalone apenas a partir dessas medidas gerais.


# Conclusão da Seção


Na fase dedicada à modelagem preditiva, foram ajustados modelos de regressão linear, tanto na forma tradicional quanto com regularização por Lasso e Ridge, com o objetivo de estimar variáveis-resposta como o peso da concha e o número de anéis.

Nessa etapa, também investigamos a possibilidade de prever características associadas ao abate, como o peso das vísceras, usando apenas medidas físicas obtidas com o animal vivo, visando reduzir intervenções invasivas e possíveis deslocamentos desnecessários dos moluscos.

Os resultados indicaram que, embora algumas variáveis explicativas exibam relações lineares razoáveis com determinados desfechos, o desempenho dos modelos permanece limitado para respostas mais complexas, especialmente o número de anéis, conforme evidenciado pelos baixos valores de $R^{2}$ e pelos elevados erros quadrático médio (EQM) e raiz do erro quadrático médio (REQM).
